In [3]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr

Useful to initiate new conda environment given the yml file (Note I adjusted it to work for windows)
```bash
conda env create -f project1_base.yml -n ml4g_project1
```

Note: Maybe we rather make a new one, since it hasn't been updated for 4 years

In [ ]:
import os
import zipfile
import shutil

# Define paths
source_dir = 'ML4G_Project_1_Data'  # Path to directory with zip files
target_dir = 'data'  # Target directory for extracted files

# Create target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Create folders for each cell line
cell_lines = ['X1', 'X2', 'X3']
for cell_line in cell_lines:
    cell_line_dir = os.path.join(target_dir, cell_line)
    os.makedirs(cell_line_dir, exist_ok=True)

# Get all zip files in the source directory (excluding sample.zip)
zip_files = [f for f in os.listdir(source_dir) if f.endswith('.zip') and f != 'sample.zip']

# Process each zip file separately
for zip_file in zip_files:
    zip_path = os.path.join(source_dir, zip_file)
    zip_name = os.path.splitext(zip_file)[0]
    print(f"Processing {zip_file}...")
    
    # Extract to a unique temporary directory for this zip file
    temp_extract_dir = os.path.join(target_dir, f'temp_{zip_name}')
    os.makedirs(temp_extract_dir, exist_ok=True)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(temp_extract_dir)
    
    # Walk through all extracted files from this zip and organize them
    for root, dirs, files in os.walk(temp_extract_dir):
        for file in files:
            source_file = os.path.join(root, file)
            
            # Handle CAGE-train files separately
            if 'CAGE-train' in root or file.endswith('.tsv'):
                # Keep CAGE-train files in their own folder
                cage_train_target = os.path.join(target_dir, 'CAGE-train')
                os.makedirs(cage_train_target, exist_ok=True)
                
                # Preserve the relative path structure for CAGE files
                rel_path = os.path.relpath(source_file, temp_extract_dir)
                target_file = os.path.join(cage_train_target, rel_path)
                os.makedirs(os.path.dirname(target_file), exist_ok=True)
                shutil.copy2(source_file, target_file)
                print(f"  Copied {file} to CAGE-train/")
                continue
            
            # Determine which cell line this file belongs to
            cell_line_found = False
            for cell_line in cell_lines:
                if cell_line in file:
                    file_lower = file.lower()
                    
                    # Get file extension (handle both .bw and .bigwig)
                    if file.endswith('.bw') or file.endswith('.bigwig'):
                        ext = '.bw'
                    elif file.endswith('.bed'):
                        ext = '.bed'
                    else:
                        continue
                    
                    # Determine the mark/assay type from the zip file name
                    zip_lower = zip_name.lower()
                    if 'dnase' in zip_lower:
                        data_type = 'DNase'
                    elif 'h3k27ac' in zip_lower:
                        data_type = 'H3K27ac'
                    elif 'h3k27me3' in zip_lower:
                        data_type = 'H3K27me3'
                    elif 'h3k36me3' in zip_lower:
                        data_type = 'H3K36me3'
                    elif 'h3k4me1' in zip_lower:
                        data_type = 'H3K4me1'
                    elif 'h3k4me3' in zip_lower:
                        data_type = 'H3K4me3'
                    elif 'h3k9me3' in zip_lower:
                        data_type = 'H3K9me3'
                    else:
                        # Skip if we can't identify the data type
                        continue
                    
                    # Create new filename: {data_type}_{cell_line}{ext}
                    new_filename = f"{data_type}_{cell_line}{ext}"
                    target_file = os.path.join(target_dir, cell_line, new_filename)
                    
                    # Copy the file to the new location
                    shutil.copy2(source_file, target_file)
                    print(f"  Copied {file} -> {cell_line}/{new_filename}")
                    cell_line_found = True
                    break
    
    # Clean up this zip's temporary directory
    shutil.rmtree(temp_extract_dir)
    print(f"  Cleaned up temp_{zip_name}")

print(f"\nAll {len(zip_files)} zip files have been extracted and organized in {target_dir}")
print(f"Structure: data/{'{X1,X2,X3}'}/{'{DataType}_{CellLine}.{bw,bed}'}")

Processing CAGE-train.zip...
  Copied X1_train_info.tsv to CAGE-train/
  Copied X1_train_y.tsv to CAGE-train/
  Copied X1_val_info.tsv to CAGE-train/
  Copied X1_val_y.tsv to CAGE-train/
  Copied X2_train_info.tsv to CAGE-train/
  Copied X2_train_y.tsv to CAGE-train/
  Copied X2_val_info.tsv to CAGE-train/
  Copied X1_train_y.tsv to CAGE-train/
  Copied X1_val_info.tsv to CAGE-train/
  Copied X1_val_y.tsv to CAGE-train/
  Copied X2_train_info.tsv to CAGE-train/
  Copied X2_train_y.tsv to CAGE-train/
  Copied X2_val_info.tsv to CAGE-train/
  Copied X2_val_y.tsv to CAGE-train/
  Copied X3_test_info.tsv to CAGE-train/
  Copied ._X1_train_info.tsv to CAGE-train/
  Copied ._X1_train_y.tsv to CAGE-train/
  Copied ._X3_test_info.tsv to CAGE-train/
  Cleaned up temp_CAGE-train
Processing DNase-bed.zip...
  Copied X2_val_y.tsv to CAGE-train/
  Copied X3_test_info.tsv to CAGE-train/
  Copied ._X1_train_info.tsv to CAGE-train/
  Copied ._X1_train_y.tsv to CAGE-train/
  Copied ._X3_test_info.tsv t

## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [6]:
# Data paths etc.
data_paths_bw = {
   'X1': {
        'DNase': 'data/X1/DNase_X1.bw',
        'H3K27ac': 'data/X1/H3K27ac_X1.bw',
        'H3K27me3': 'data/X1/H3K27me3_X1.bw',
        'H3K36me3': 'data/X1/H3K36me3_X1.bw',
        'H3K4me1': 'data/X1/H3K4me1_X1.bw',
        'H3K4me3': 'data/X1/H3K4me3_X1.bw',
        'H3K9me3': 'data/X1/H3K9me3_X1.bw'
    },
    'X2': {
        'DNase': 'data/X2/DNase_X2.bw',
        'H3K27ac': 'data/X2/H3K27ac_X2.bw',
        'H3K27me3': 'data/X2/H3K27me3_X2.bw',
        'H3K36me3': 'data/X2/H3K36me3_X2.bw',
        'H3K4me1': 'data/X2/H3K4me1_X2.bw',
        'H3K4me3': 'data/X2/H3K4me3_X2.bw',
        'H3K9me3': 'data/X2/H3K9me3_X2.bw'
    },
    'X3': {
        'DNase': 'data/X3/DNase_X3.bw',
        'H3K27ac': 'data/X3/H3K27ac_X3.bw',
        'H3K27me3': 'data/X3/H3K27me3_X3.bw',
        'H3K36me3': 'data/X3/H3K36me3_X3.bw',
        'H3K4me1': 'data/X3/H3K4me1_X3.bw',
        'H3K4me3': 'data/X3/H3K4me3_X3.bw',
        'H3K9me3': 'data/X3/H3K9me3_X3.bw'
    }
}

data_paths_bed = {
     'X1': {
        'DNase': 'data/X1/DNase_X1.bed',
        'H3K27ac': 'data/X1/H3K27ac_X1.bed',
        'H3K27me3': 'data/X1/H3K27me3_X1.bed',
        'H3K36me3': 'data/X1/H3K36me3_X1.bed',
        'H3K4me1': 'data/X1/H3K4me1_X1.bed',
        'H3K4me3': 'data/X1/H3K4me3_X1.bed',
        'H3K9me3': 'data/X1/H3K9me3_X1.bed'
    },
    'X2': {
        'DNase': 'data/X2/DNase_X2.bed',
        'H3K27ac': 'data/X2/H3K27ac_X2.bed',
        'H3K27me3': 'data/X2/H3K27me3_X2.bed',
        'H3K36me3': 'data/X2/H3K36me3_X2.bed',
        'H3K4me1': 'data/X2/H3K4me1_X2.bed',
        'H3K4me3': 'data/X2/H3K4me3_X2.bed',
        'H3K9me3': 'data/X2/H3K9me3_X2.bed'
    },
    'X3': {
        'DNase': 'data/X3/DNase_X3.bed',
        'H3K27ac': 'data/X3/H3K27ac_X3.bed',
        'H3K27me3': 'data/X3/H3K27me3_X3.bed',
        'H3K36me3': 'data/X3/H3K36me3_X3.bed',
        'H3K4me1': 'data/X3/H3K4me1_X3.bed',
        'H3K4me3': 'data/X3/H3K4me3_X3.bed',
        'H3K9me3': 'data/X3/H3K9me3_X3.bed'
    }
}

gene_paths = {
    'X1': {
        'train':{
            'info': 'data/CAGE-train/CAGE-train/X1_train_info.tsv',
            'target': 'data/CAGE-train/CAGE-train/X1_train_y.tsv'
        },
        'validation':{
            'info': 'data/CAGE-train/CAGE-train/X1_val_info.tsv',
            'target': 'data/CAGE-train/CAGE-train/X1_val_y.tsv'
        }
    },
    'X2': {
        'train':{
            'info': 'data/CAGE-train/CAGE-train/X2_train_info.tsv',
            'target': 'data/CAGE-train/CAGE-train/X2_train_y.tsv'
        },
        'validation':{
            'info': 'data/CAGE-train/CAGE-train/X2_val_info.tsv',
            'target': 'data/CAGE-train/CAGE-train/X2_val_y.tsv'
        }
    },
    'X3': 'X3_test_info.tsv'
}

In [9]:
# Approach 1: Use bw files, compute average, min, max, std over a region that spans the transcription start site (TSS) +/- 10kb
import pyBigWig

# Idea: Function that given cell line and type (train/val) extracts features and targets
def get_dataset(cell_line, set_type):

    gene_info_df = pd.read_csv(gene_paths[cell_line][set_type]['info'], sep='\t')
    gene_target_df = pd.read_csv(gene_paths[cell_line][set_type]['target'], sep='\t')
    
    features = extract_all_features(gene_info_df, cell_line)
    targets = gene_target_df['gex'].values

    return features, targets

def extract_all_features(gene_info_df, cell_line, window=10000, bw_paths=None):
    """
    Extract summary statistics from bigWig files for each gene in `gene_info_df` and
    each mark defined in `bw_paths` (or `data_paths_bw[cell_line]` by default).

    Parameters:
    - gene_info_df: pd.DataFrame with at least columns ['chr', 'tss'] and optionally 'gene_name'.
    - cell_line: one of the keys in data_paths_bw (e.g. 'X1').
    - window: int, number of bases upstream/downstream of TSS to include (default 10_000).
    - bw_paths: optional dict of mark->path. If None, uses data_paths_bw[cell_line].

    Returns:
    - pd.DataFrame: index matches gene order (uses 'gene_name' if present), columns like '<MARK>_avg', '<MARK>_min', '<MARK>_max', '<MARK>_std'.
    """
    if bw_paths is None:
        bw_paths = data_paths_bw.get(cell_line, {})

    # Prepare index / gene identifiers
    if 'gene_name' in gene_info_df.columns:
        gene_names = gene_info_df['gene_name'].astype(str).tolist()
    else:
        gene_names = gene_info_df.index.astype(str).tolist()

    feature_frames = []

    for mark, path in bw_paths.items():
        print(f"Extracting features for {mark} from {path}...")

        # Default fallback columns for this mark (will keep gene order)
        cols = [f"{mark}_avg", f"{mark}_min", f"{mark}_max", f"{mark}_std"]
        rows = []

        # Check file exists
        try:
            bw = pyBigWig.open(path)
            if bw is None:
                raise FileNotFoundError(f"Could not open bigWig: {path}")
        except Exception as e:
            print(f"  Warning: cannot open {path}: {e}. Filling zeros for this mark.")
            # fill zeros for all genes if the file can't be opened
            rows = [[0.0, 0.0, 0.0, 0.0] for _ in range(len(gene_names))]
            feature_frames.append(pd.DataFrame(rows, columns=cols, index=gene_names))
            continue

        # iterate genes in order and extract statistics
        for _, row in gene_info_df.iterrows():
            chrom = str(row['chr'])
            
            start = max(0, int(row['TSS_start']) - window)
            end = int(row['TSS_end']) + window
            try:
                values = bw.values(chrom, start, end, numpy=True)
                # pyBigWig may return None if region is invalid
                if values is None:
                    vals = np.array([], dtype=float)
                else:
                    vals = np.asarray(values, dtype=float)
                # replace NaNs and handle empty arrays
                if vals.size == 0:
                    rows.append([0.0, 0.0, 0.0, 0.0])
                else:
                    vals = np.nan_to_num(vals, nan=0.0)
                    rows.append([float(np.mean(vals)), float(np.min(vals)), float(np.max(vals)), float(np.std(vals))])
            except Exception as e:
                # keep alignment with genes even when extraction fails for a gene
                print(f"  Warning: failed for {chrom}:{start}-{end} ({e}). Using zeros.")
                rows.append([0.0, 0.0, 0.0, 0.0])

        # close bigWig handle
        try:
            bw.close()
        except Exception:
            pass

        # create dataframe for this mark, use gene_names as index to preserve order
        mark_df = pd.DataFrame(rows, columns=cols, index=gene_names)
        feature_frames.append(mark_df)

    # concat horizontally and ensure index is gene names
    if feature_frames:
        all_features_df = pd.concat(feature_frames, axis=1)
        all_features_df.index.name = 'gene_name'
    else:
        all_features_df = pd.DataFrame(index=gene_names)

    return all_features_df

In [ ]:
X, y = get_dataset('X1', 'train')

Extracting features for DNase from data/X1/DNase_X1.bw...
Extracting features for H3K27ac from data/X1/H3K27ac_X1.bw...
Extracting features for H3K27me3 from data/X1/H3K27me3_X1.bw...
Extracting features for H3K36me3 from data/X1/H3K36me3_X1.bw...
Extracting features for H3K4me1 from data/X1/H3K4me1_X1.bw...
Extracting features for H3K4me3 from data/X1/H3K4me3_X1.bw...
Extracting features for H3K9me3 from data/X1/H3K9me3_X1.bw...
           DNase_avg  DNase_min  DNase_max  DNase_std  H3K27ac_avg  \
gene_name                                                            
SLC20A1     0.120745   0.024102   0.929387   0.114712     1.100506   
C11orf58    0.145684   0.024102   4.175937   0.437137     4.124743   
ZSCAN9      0.114666   0.024102   2.427793   0.232137     1.081377   
CD19        0.355132   0.024102   3.083350   0.541315     9.234150   
TMEM123     0.202145   0.024102   2.615094   0.321339     2.547052   

           H3K27ac_min  H3K27ac_max  H3K27ac_std  H3K27me3_avg  H3K27me3_mi

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [34]:
def get_data_dict():
    data_dict = {
        'X1': {
            'train': get_dataset('X1', 'train'),
            'validation': get_dataset('X1', 'validation')
        },
        'X2': {
            'train': get_dataset('X2', 'train'),
            'validation': get_dataset('X2', 'validation')
        }
    }
    return data_dict


def evaluate_model(model, data_dict):
    """ The pipeline goes something like this:
    1) We get all the data (X1 train, X1 Validation, X2 train, X2 validation)
    2) Then we try the following combinations:
        a) Train model on X1 train, validate on X1 validation and X2 validation
        b) Train model on X2 train, validate on X2 validation and X1 validation
        c) Train model on X1 + X2 train, validate on X1 validation and X2 validation    
    """

    results = {}
    scaler = StandardScaler()

    for train_cell_line in ['X1', 'X2', ['X1', 'X2']]:
        # Train on single cell line
        if isinstance(train_cell_line, list):
            X_train = np.concatenate([data_dict[cell]['train'][0] for cell in train_cell_line])
            y_train = np.concatenate([data_dict[cell]['train'][1] for cell in train_cell_line])
        else:
            X_train, y_train = data_dict[train_cell_line]['train']
        
        model.fit(scaler.fit_transform(X_train), y_train)
        
        for val_cell_line in ['X1', 'X2']:
            X_val, y_val = data_dict[val_cell_line]['validation']
            X_val_scaled = scaler.transform(X_val)
            y_val_pred = model.predict(X_val_scaled)
            
            mse = mean_squared_error(y_val, y_val_pred)
            r2 = r2_score(y_val, y_val_pred)
            spearmanr_corr, _ = spearmanr(y_val, y_val_pred)

            results[(str(train_cell_line), str(val_cell_line))] = {
                'MSE': mse,
                'R^2': r2,
                "Spearman's rho": spearmanr_corr
            }

    return results

In [22]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

In [ ]:
data_dict = get_data_dict()

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
}


Extracting features for DNase from data/X1/DNase_X1.bw...
Extracting features for H3K27ac from data/X1/H3K27ac_X1.bw...
Extracting features for H3K27me3 from data/X1/H3K27me3_X1.bw...
Extracting features for H3K36me3 from data/X1/H3K36me3_X1.bw...
Extracting features for H3K4me1 from data/X1/H3K4me1_X1.bw...
Extracting features for H3K4me3 from data/X1/H3K4me3_X1.bw...
Extracting features for H3K9me3 from data/X1/H3K9me3_X1.bw...
Extracting features for DNase from data/X1/DNase_X1.bw...
Extracting features for H3K27ac from data/X1/H3K27ac_X1.bw...
Extracting features for H3K27me3 from data/X1/H3K27me3_X1.bw...
Extracting features for H3K36me3 from data/X1/H3K36me3_X1.bw...
Extracting features for H3K4me1 from data/X1/H3K4me1_X1.bw...
Extracting features for H3K4me3 from data/X1/H3K4me3_X1.bw...
Extracting features for H3K9me3 from data/X1/H3K9me3_X1.bw...
Extracting features for DNase from data/X2/DNase_X2.bw...
Extracting features for H3K27ac from data/X2/H3K27ac_X2.bw...
Extracting f

/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


TypeError: unhashable type: 'list'

In [36]:

for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    results = evaluate_model(model, data_dict)
    for (train_cell_line, val_cell_line), metrics in results.items():
        mse = metrics["MSE"]
        r2 = metrics["R^2"]
        spearman_rho = metrics["Spearman's rho"]
        print(f"  Trained on {train_cell_line}, validated on {val_cell_line}: MSE={mse:.4f}, R^2={r2:.4f}, Spearman's rho={spearman_rho:.4f}")
    print()

Evaluating Linear Regression...
  Trained on X1, validated on X1: MSE=82660.0225, R^2=0.0473, Spearman's rho=0.5504
  Trained on X1, validated on X2: MSE=72527.6524, R^2=0.0060, Spearman's rho=0.4331
  Trained on X2, validated on X1: MSE=83889.4926, R^2=0.0332, Spearman's rho=0.4691
  Trained on X2, validated on X2: MSE=68014.3090, R^2=0.0678, Spearman's rho=0.4598
  Trained on ['X1', 'X2'], validated on X1: MSE=81185.9989, R^2=0.0643, Spearman's rho=0.5453
  Trained on ['X1', 'X2'], validated on X2: MSE=68727.0658, R^2=0.0581, Spearman's rho=0.4996

Evaluating Ridge Regression...
  Trained on X1, validated on X1: MSE=82659.6599, R^2=0.0473, Spearman's rho=0.5504
  Trained on X1, validated on X2: MSE=72530.9050, R^2=0.0059, Spearman's rho=0.4333
  Trained on X2, validated on X1: MSE=83874.1543, R^2=0.0333, Spearman's rho=0.4694
  Trained on X2, validated on X2: MSE=68015.2145, R^2=0.0678, Spearman's rho=0.4600
  Trained on ['X1', 'X2'], validated on X1: MSE=81184.6267, R^2=0.0643, Spea

/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home

  Trained on X1, validated on X1: MSE=82658.2743, R^2=0.0474, Spearman's rho=0.5505
  Trained on X1, validated on X2: MSE=72411.3779, R^2=0.0076, Spearman's rho=0.4453
  Trained on X2, validated on X1: MSE=83120.2717, R^2=0.0420, Spearman's rho=0.4762
  Trained on X2, validated on X2: MSE=68075.3723, R^2=0.0670, Spearman's rho=0.4646
  Trained on ['X1', 'X2'], validated on X1: MSE=81094.6037, R^2=0.0654, Spearman's rho=0.5481
  Trained on ['X1', 'X2'], validated on X2: MSE=68744.2164, R^2=0.0578, Spearman's rho=0.5059

Evaluating Random Forest...
  Trained on X1, validated on X1: MSE=113119.3937, R^2=-0.3037, Spearman's rho=0.5837
  Trained on X1, validated on X2: MSE=500145.7494, R^2=-5.8546, Spearman's rho=0.4877
  Trained on X2, validated on X1: MSE=526814.3827, R^2=-5.0716, Spearman's rho=0.5657
  Trained on X2, validated on X2: MSE=79335.4166, R^2=-0.0873, Spearman's rho=0.5701
  Trained on ['X1', 'X2'], validated on X1: MSE=93014.0180, R^2=-0.0720, Spearman's rho=0.5853
  Trained

/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/janic/miniconda3/envs/ml4g/lib/python3.10/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [ ]:
# TODO: 
# Load your feature (bed and/or bigwig and/or fasta) and target files (tsv) here.
# Decide which features to use for training. Feel free to process them however you need.

# NOTE: 
# bed and bigwig files contain signals of all chromosomes (including sex chromosomes).
# Training and validation split based on chromosomes has been done for you. 
# However, you can resplit the data in any way you want.

path_data = "data\\CAGE-train\\CAGE-train" 
path_test = "data\\CAGE-train\\CAGE-train\\X3_test_info.tsv"
test_genes = pd.read_csv(path_test, sep='\t')
# ---------------------------INSERT CODE HERE---------------------------




# ---------------------------------------------------------------------- 

## Work Package 1.2 - Model Building

In [ ]:
# TODO: 
# Select the best model to predict gene expression from the obtained features in WP 1.1.

# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [ ]:
# TODO:
# Using the model trained in WP 1.2, make predictions on the test data (chr 1 of cell line X3).
# Store predictions in a variable called "pred" which is a numpy array.
pred: np.ndarray
pred = np.array([])  # TODO
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

#### Store Predictions in the Required Format

In [ ]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'path/to/save/output/file'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "LastName_FirstName_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes['gex_predicted'] = pred.tolist()
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)